<a href="https://colab.research.google.com/github/Uniholder/DeepLearningSchool/blob/main/2_semester/1_NLP/%5Bhomework%5Dsimple_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
# !pip install --upgrade gensim

In [1]:
import os

PATH_TO_DATA = 'data/'

In [8]:
!wget -O $PATH_TO_DATA/SO_vectors_200 https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-10-05 13:13:52--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘data//SO_vectors_200’

data//SO_vectors_20 100%[===================>]   1.35G  20.4MB/s    in 2m 51s  

2021-10-05 13:16:44 (8.13 MB/s) - ‘data//SO_vectors_200’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format(os.path.join(PATH_TO_DATA, 'SO_vectors_200'), binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# method most_simmilar
wv_embeddings.most_similar(positive='dog', topn=20)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885),
 ('canine', 0.7209057211875916),
 ('meow', 0.7185295820236206),
 ('cow', 0.7080444693565369),
 ('dog2', 0.7057910561561584),
 ('woof', 0.705061137676239),
 ('dog1', 0.7038840651512146),
 ('dog3', 0.701882004737854),
 ('penguin', 0.697029173374176),
 ('bulldog', 0.6940488815307617),
 ('mammals', 0.6931389570236206)]

In [ ]:
# косинусная схожесть между dog и cat
wv_embeddings.similarity('dog', 'cat')

0.6852341

**Ответ:**

Слово cat не входит в топ-5 близких к dog. Входит cats (множественное число).

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [3]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer, word_tokenize
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
#         return word_tokenize(text)
tokenizer = MyTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    question: строка
    embeddings: наше векторное представление
    dim: размер любого вектора в нашем представлении
        
    return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question)
    v_question = np.zeros(dim)
    count = 0
    for token in tokens:
        try:
            v_question += embeddings[token]
            count += 1
        except KeyError:
            pass
        
    return v_question / count if count != 0 else v_question

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
sentence = 'I love neural networks'

v_sentence = question_to_vec(sentence, wv_embeddings, tokenizer)
v_sentence[:10]

array([-1.01422749, -1.68912624, -1.28541217, -1.37103013,  0.15916634,
        0.0670696 ,  1.11884137, -0.07051762,  0.53863693, -0.37139114])

**Ответ:**

In [ ]:
round(v_sentence[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

**Решение:**

Заменим для упрощения $rank\_q_i^{'}$ на $x_i$.

$max(\frac{1}{N}\sum_{i=1}^N \, [x_i \le 47] - \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+x_i)}\cdot[x_i \le 1]) = \\
\frac{1}{N} [max(\sum_{i=1}^N \, [x_i \le 47]) - min(\sum_{i=1}^N\frac{1}{\log_2(1+x_i)}\cdot[x_i \le 1])]$

1. $max(\sum_{i=1}^N \, [x_i \le 47])$

Максимум достигается при значениях $x_i \in [1; 47]$ для всех $i \in [1; N]$

$\sum_{i=1}^N \ 1 = N$

2. $min(\sum_{i=1}^N\frac{1}{\log_2(1+x_i)}\cdot[x_i \le 1])$

Минимум достигается при значениях $x_i \in [2, \infty)$ для всех $i \in [1; N]$

$\sum_{i=1}^N\frac{1}{\log_2(1+x_i)}\cdot 0 = 0$

Таким образом получаем:

$\frac{1}{N}(N - 0) = 1$

**Ответ:**

1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
1 / np.log2(10)

0.3010299956639812

**Ответ:**

$\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}} \approx 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [32]:
def hits_count(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов
    result: вернуть  Hits@k
    """
    hits_value = 0
    for rank in dup_ranks:
        hits_value += (rank <= k)
    
    return hits_value / len(dup_ranks)

In [33]:
def dcg_score(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов
    result: вернуть DCG@k
    """
    dcg_value = 0
    for rank in dup_ranks:
        dcg_value += (rank <= k) / np.log2(1 + rank)
    
    return dcg_value / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
# !gdown --id 1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_

In [ ]:
# !unzip data/stackoverflow_similar_questions.zip -d data/

Считайте данные.

In [4]:
import os

PATH_TO_DATA = 'data/data/'

In [5]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as f:
        for line in f.readlines():
            data.append(line.strip())
    return data

Нам понадобится только файл validation.

In [6]:
validation_data = read_corpus(os.path.join(PATH_TO_DATA, 'validation.tsv'))

Кол-во строк

In [7]:
len(validation_data)

3760

In [8]:
len(validation_data[0].split('\t'))

1001

In [9]:
validation_data[0].split('\t')[1000]

'Eclipse: Most useful refactorings'

Размер нескольких первых строк

In [10]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54516
2 52817
3 54463
4 53723
5 52296


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy
from scipy.spatial.distance import cosine

import nltk
from nltk.stem import PorterStemmer
import spacy

In [12]:
nlp = spacy.load('en_core_web_sm')

In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    используем стеммер портера
    question: строка
    embeddings: наше векторное представление
    dim: размер любого вектора в нашем представлении
        
    return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question)
    
    # ps = PorterStemmer()
    # tokens = list(map(ps.stem, tokens))

    # очень долго
#     spacy_results = nlp(question)
#     tokens = ' '.join([token.lemma_ for token in spacy_results])

#     wnl = nltk.WordNetLemmatizer()
#     tokens = ' '.join([wnl.lemmatize(word) for word in tokens])
    
    v_question = np.zeros(dim)
    count = 0
    for token in tokens:
        try:
            v_question += embeddings[token]
            count += 1
        except KeyError:
            pass
        
    return v_question / count if count != 0 else v_question

In [15]:
def rank_candidates_scipy(question, candidates, embeddings, tokenizer, dim=200, return_cosine=False):
    """
    для вычисления похожести используем scipy 
    question: строка
    candidates: массив строк(кандидатов) [a, b, c]
    result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    v_question = question_to_vec(question, embeddings, tokenizer)
    
    def candidates_cos(candidate):
        v_candidate = question_to_vec(candidate, embeddings, tokenizer, dim=dim)
        return 1 - cosine(v_question, v_candidate)
    
    result = []
    if return_cosine:
        for i, candidate in enumerate(candidates):
            result.append((i, candidate, candidates_cos(candidate)))
    else:
        for i, candidate in enumerate(candidates):
            result.append((i, candidate))
    
    return sorted(result, key=lambda x: candidates_cos(x[1]), reverse=True)

In [16]:
def rank_candidates_sklearn(question, candidates, embeddings, tokenizer, dim=200, return_cosine=False):
    """
    для вычисления похожести используем sklearn
    question: строка
    candidates: массив строк(кандидатов) [a, b, c]
    result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    v_question = question_to_vec(question, embeddings, tokenizer)
    v_candidates = [question_to_vec(candidate, embeddings, tokenizer, dim=dim) for candidate in candidates]
    similarities = cosine_similarity(v_question.reshape(1, -1), v_candidates).squeeze()
    
    result = sorted(
        list(zip(range(len(candidates)), candidates, similarities)), 
        key=lambda x: x[2], 
        reverse=True
    )
    
    return result if return_cosine else [(i, c) for i, c, _ in result]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [17]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates_scipy(question, q_candidates, wv_embeddings, tokenizer, return_cosine=True)
    print(ranks)
    print()

[(1, 'C# create cookie from string and send it', 0.5630749321491995), (0, 'Convert Google results object (pure js) to Python object', 0.47886280839497886), (2, 'How to use jQuery AJAX for an outside domain?', 0.161884531003609)]

[(1, 'WPF- How to update the changes in list item of a list', 0.48369789343599146), (0, 'Getting all list items of an unordered list in PHP', 0.4569547639204963), (2, 'select2 not displaying search results', 0.16329035032381012)]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ:**

- 102
- 021 (стемминг)
- 201 (лемматизация)

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [29]:
from tqdm.notebook import tqdm

In [39]:
%%time
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line.split('\t')
    ranks = rank_candidates_scipy(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


CPU times: user 1min 29s, sys: 1.1 s, total: 1min 30s
Wall time: 1min 30s


In [40]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.495 | Hits@   1: 0.495
DCG@   5: 0.574 | Hits@   5: 0.642
DCG@  10: 0.602 | Hits@  10: 0.729
DCG@ 100: 0.661 | Hits@ 100: 0.999
DCG@ 500: 0.661 | Hits@ 500: 1.000
DCG@1000: 0.661 | Hits@1000: 1.000


sklearn.metrics.pairwise.cosine_similarity<br>-|-<br>-|-<br>PosterStemming|-<br>-|-<br>WordNetLemmatizer|-<br>-|
-|-|-|-|-|-|
DCG@   1: 0.276 | Hits@   1: 0.276|DCG@   1: 0.324 | Hits@   1: 0.324|DCG@   1: 0.050 | Hits@   1: 0.050
DCG@   5: 0.332 | Hits@   5: 0.386|DCG@   5: 0.398 | Hits@   5: 0.466|DCG@   5: 0.074 | Hits@   5: 0.098
DCG@  10: 0.349 | Hits@  10: 0.438|DCG@  10: 0.422 | Hits@  10: 0.538|DCG@  10: 0.086 | Hits@  10: 0.135
DCG@ 100: 0.395 | Hits@ 100: 0.671|DCG@ 100: 0.474 | Hits@ 100: 0.797|DCG@ 100: 0.135 | Hits@ 100: 0.388
DCG@ 500: 0.421 | Hits@ 500: 0.875|DCG@ 500: 0.493 | Hits@ 500: 0.949|DCG@ 500: 0.179 | Hits@ 500: 0.736
DCG@1000: 0.434 | Hits@1000: 1.000|DCG@1000: 0.499 | Hits@1000: 1.000|DCG@1000: 0.207 | Hits@1000: 1.000

sklearn.metrics.pairwise.cosine_similarity<br>PosterStemming<br>MyTockenizer|-<br>-<br>-
-|-
DCG@   1: 0.333 | Hits@   1: 0.333
DCG@   5: 0.406 | Hits@   5: 0.474
DCG@  10: 0.426 | Hits@  10: 0.536
DCG@ 100: 0.478 | Hits@ 100: 0.793
DCG@ 500: 0.499 | Hits@ 500: 0.949
DCG@1000: 0.504 | Hits@1000: 1.000

scipy.spatial.distance.cosine<br>-|-<br>-|-<br>PosterStemming|-<br>-|-<br>WordNetLemmatizer|-<br>-
-|-|-|-|-|-
DCG@   1: 0.463 | Hits@   1: 0.463|DCG@   1: 0.325 | Hits@   1: 0.325|DCG@   1: 0.050 | Hits@   1: 0.050
DCG@   5: 0.539 | Hits@   5: 0.609|DCG@   5: 0.405 | Hits@   5: 0.479|DCG@   5: 0.074 | Hits@   5: 0.098
DCG@  10: 0.571 | Hits@  10: 0.708|DCG@  10: 0.426 | Hits@  10: 0.544|DCG@  10: 0.086 | Hits@  10: 0.136
DCG@ 100: 0.633 | Hits@ 100: 0.997|DCG@ 100: 0.480 | Hits@ 100: 0.813|DCG@ 100: 0.135 | Hits@ 100: 0.389
DCG@ 500: 0.633 | Hits@ 500: 1.000|DCG@ 500: 0.499 | Hits@ 500: 0.960|DCG@ 500: 0.180 | Hits@ 500: 0.740
DCG@1000: 0.633 | Hits@1000: 1.000|DCG@1000: 0.503 | Hits@1000: 1.000|DCG@1000: 0.207 | Hits@1000: 1.000

scipy.spatial.distance.cosine<br>MyTockenizer|-<br>-
-|-
DCG@   1: 0.495 | Hits@   1: 0.495
DCG@   5: 0.574 | Hits@   5: 0.642
DCG@  10: 0.602 | Hits@  10: 0.729
DCG@ 100: 0.661 | Hits@ 100: 0.999
DCG@ 500: 0.661 | Hits@ 500: 1.000
DCG@1000: 0.661 | Hits@1000: 1.000

### Эмбеддинги, обученные на корпусе похожих вопросов

In [20]:
train_data = read_corpus(os.path.join(PATH_TO_DATA, 'train.tsv'))

In [21]:
train_data[1].split('\t')

['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
 'Event handling for geometries in Three.js?']

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [22]:
%%time
words = []
for row in train_data:
    line = ' '.join(row.split('\t'))
    tokens = tokenizer.tokenize(line)
    words.append(tokens)

CPU times: user 9.36 s, sys: 1.41 s, total: 10.8 s
Wall time: 10.8 s


In [23]:
len(words)

1000000

In [52]:
%%time
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=20).wv

CPU times: user 7min 30s, sys: 1.4 s, total: 7min 31s
Wall time: 3min 55s


In [53]:
len(embeddings_trained)

50830

In [54]:
len(wv_embeddings)

1787145

In [27]:
len(validation_data)

3760

In [55]:
%%time
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line.split('\t')
    ranks = rank_candidates_scipy(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


CPU times: user 1min 33s, sys: 1.18 s, total: 1min 34s
Wall time: 1min 34s


In [56]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.301 | Hits@   1: 0.301
DCG@   5: 0.372 | Hits@   5: 0.431
DCG@  10: 0.394 | Hits@  10: 0.501
DCG@ 100: 0.448 | Hits@ 100: 0.766
DCG@ 500: 0.470 | Hits@ 500: 0.936
DCG@1000: 0.477 | Hits@1000: 1.000


sklearn.metrics.pairwise.cosine_similarity<br>PosterStemming<br>MyTockenizer|-<br>-<br>-
-|-
DCG@   1: 0.202 | Hits@   1: 0.202
DCG@   5: 0.256 | Hits@   5: 0.302
DCG@  10: 0.271 | Hits@  10: 0.350
DCG@ 100: 0.321 | Hits@ 100: 0.596
DCG@ 500: 0.353 | Hits@ 500: 0.853
DCG@1000: 0.369 | Hits@1000: 1.000

scipy.spatial.distance.cosine<br>MyTockenizer|-<br>-
-|-
DCG@   1: 0.255 | Hits@   1: 0.255
DCG@   5: 0.314 | Hits@   5: 0.370
DCG@  10: 0.338 | Hits@  10: 0.443
DCG@ 100: 0.390 | Hits@ 100: 0.698
DCG@ 500: 0.417 | Hits@ 500: 0.911
DCG@1000: 0.427 | Hits@1000: 1.000

scipy.spatial.distance.cosine<br>MyTockenizer<br>window=20|-<br>-<br>-
-|-
DCG@   1: 0.301 | Hits@   1: 0.301
DCG@   5: 0.372 | Hits@   5: 0.431
DCG@  10: 0.394 | Hits@  10: 0.501
DCG@ 100: 0.448 | Hits@ 100: 0.766
DCG@ 500: 0.470 | Hits@ 500: 0.936
DCG@1000: 0.477 | Hits@1000: 1.000

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. Токенизация

Были рассмотрены типы:
- слово
- слово + пунктуация
2. Нормализация

Зависит от библитеки подсчёта векторного расстояния
3. Эмбеддинги

Преробученный лучше
4. Качество

Мало данных
5. Подход

Больше данных для обучения эмбеддингов